In [ ]:
# Montar directorio de drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git


In [ ]:
!pip install python-dotenv

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('Hay %d GPU(s) disponibles.' % torch.cuda.device_count())
    print('Vamos a usar la GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No hay GPU disponible, usando CPU.')

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from trl import SFTTrainer

from huggingface_hub import login
from transformers import TrainingArguments, EarlyStoppingCallback, AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import wandb
import pandas as pd
from datasets import Dataset
from dotenv import load_dotenv
import os
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, average_precision_score, accuracy_score, precision_recall_fscore_support, precision_score, recall_score

In [ ]:
path = '/content/drive/Othercomputers/Mi PC/TFG/MySonGyny/MySonGyny-2025'
#path = '../'
#path = '/home/lobezno/Documentos'

In [ ]:
load_dotenv(f'{path}/cuadernos/.env')  # Carga las variables del .env

HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
WANDB_TOKEN = os.getenv("WANDB_TOKEN")

In [ ]:
hf_token = HUGGINGFACE_TOKEN
login(hf_token)

In [ ]:
wb_token = WANDB_TOKEN

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune-con-CoT-Tarea-2',
    job_type="training",
    anonymous="allow"
)

In [ ]:
COLUMN_ID = 'id'
COLUMN_LETRA = 'lyrics'
COLUMN_ETIQUETA = 'label'
COLUNM_ETIQUETALETRA = 'labelconletras'
COLUMN_RAZONAMIENTO = 'reasoning'
tipo_CoT = 'GPT4o'
model_name = 'DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit'
csv_train_filepath = f'{path}/data/train_data/task2_GPT4o_dataReasoning.csv'
tipo_prompt = 0
#csv_test_filepath = f'{path}/data/train_data/testData.csv'

In [ ]:
print(f"Cargando datos desde: {csv_train_filepath}")
try:
    # 1. Cargar el DataFrame separado
    df = pd.read_csv(csv_train_filepath, encoding='utf-8', encoding_errors="replace", sep=',')
    #test_df = pd.read_csv(csv_test_filepath, encoding='utf-8', encoding_errors="replace", sep=',')
    print(f"Columnas encontradas en el CSV: {df.columns.tolist()}")

    # 2. Dividir los datos
    print(f"Total de filas válidas antes de dividir: {len(df)}")
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[COLUMN_ETIQUETA])
    train_df, valid_df = train_test_split(train_df, test_size=0.3, random_state=42, stratify=train_df[COLUMN_ETIQUETA])
    print(f"División completada:")
    print(f"  - Tamaño del conjunto de Entrenamiento: {len(train_df)}")
    print(f"  - Tamaño del conjunto de Validación:  {len(valid_df)}")
    print(f"  - Tamaño del conjunto de Test:         {len(test_df)}")

     # --- COMPROBACIÓN DE PROPORCIONES DE CLASE ---
    print("\n--- Distribución de Clases ---")
    print("Conjunto de Entrenamiento:")
    print(train_df[COLUNM_ETIQUETALETRA].value_counts(normalize=False)) # Conteos absolutos

    print("\nConjunto de Validación:")
    print(valid_df[COLUNM_ETIQUETALETRA].value_counts(normalize=False))

    print("\nConjunto de Test:")
    print(test_df[COLUNM_ETIQUETALETRA].value_counts(normalize=False))
    print("-----------------------------")

    nr_class_df = train_df[train_df[COLUMN_ETIQUETA] == 0]
    s_class_df = train_df[train_df[COLUMN_ETIQUETA] == 1]
    v_class_df = train_df[train_df[COLUMN_ETIQUETA] == 2]
    h_class_df = train_df[train_df[COLUMN_ETIQUETA] == 3]

    #----PARA HACER OVERSAMPLING---
    v_oversampled_df = resample(v_class_df,
                                       replace=True,     # Permitir duplicados
                                       n_samples=198,
                                       random_state=42)
    h_oversampled_df = resample(h_class_df,
                                       replace=True,     # Permitir duplicados
                                       n_samples=156,
                                       random_state=42)

    train_df_balanced = pd.concat([nr_class_df, s_class_df, v_oversampled_df, h_oversampled_df])

    #----PARA HACER UNDERSAMPLING---
    #majority_undersampled_df = resample(majority_class_df,
    #                                replace=False,  # Sin duplicados
    #                                n_samples=600,  # Mismo tamaño que minoritaria
    #                                random_state=42)

    #train_df_balanced = pd.concat([majority_undersampled_df, minority_oversampled_df])


    # Mezclar el DataFrame resultante para que las clases no estén agrupadas
    train_df_balanced = train_df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

    print("\nConjunto de Entrenamiento nuevo:")
    print(train_df_balanced[COLUNM_ETIQUETALETRA].value_counts(normalize=False))
    print("------------------------------------------")

    # 3. Convertir a datasets.Dataset
    trainData = Dataset.from_pandas(train_df_balanced)
    validData = Dataset.from_pandas(valid_df)
    testData = Dataset.from_pandas(test_df) # Guarda el de test también

    print("\nDatasets creados exitosamente.")
    print(f"Columnas en trainData: {trainData.column_names}")
except FileNotFoundError:
    print(f"Error: Archivo no encontrado en '{csv_train_filepath}'")
except Exception as e:
    print(f"Error inesperado durante la carga o división de datos: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"unsloth/{model_name}",  # Nombre del modelo en Hugging Face (modelo LLaMA distilado y más ligero)
    max_seq_length = max_seq_length,                      # Longitud máxima de secuencia que puede procesar (por ejemplo 2048 o 4096 tokens)
    dtype = dtype,                                        # Tipo de precisión para los cálculos (por ejemplo: torch.float16 o torch.bfloat16)
)

# Prueba con el modelo sin entrenar

In [ ]:
promp_style = """### Instruccion
            Analiza la siguiente letra de canción y determina si contiene contenido misógino. Evalúa si incluye lenguaje, actitudes o mensajes que:
              - Degraden o deshumanicen a las mujeres.
              - Menosprecien a las mujeres de manera explícita o implícita.
              - Refuercen estereotipos negativos o dañinos sobre las mujeres.
              - Promuevan violencia física, emocional o sexual contra las mujeres.

            Primero, piensa paso a paso tu razonamiento.
            DESPUÉS de tu razonamiento interno, responde ÚNICA Y EXCLUSIVAMENTE con "1" si la letra es misógina o con "0" si la letra no es misógina. Tu respuesta final debe ser solo un dígito.

            ### Letra:
            {}

            ### Respuesta: <think>"""

In [ ]:
cancion = """a de nuevo solos tú y yo, un lago y una canción echo de menos oír tu voz, una estrella te eclipsó los momentos que no volverán a sentir tu piel en mis brazos te tuve ayer hay tantas cosas que te quiero decir, acércate a veces siento al despertar como un susurro, tu calor ella no deja de pensar que un día te encontrará cógeme no me dejes marchar, quiero sentarme a tus pies en mis brazos te tuve ayer hay tantas cosas que te quiero decir, dime por que sólo tú, tú y yo una guitarra, el lago y una canción sólo tú, tú y yo ahora ya te puedo decir ¡adió-oh-oh-os! ah-ah-ah-ah ah-ah-ah-ah de nuevo solos tú y yo un lago y una canción echo de menos oír tu voz una estrella te eclipsó"""

In [ ]:
import re
# Prepara el modelo para la inferencia (modo generación, sin entrenamiento)
FastLanguageModel.for_inference(model)

# Formatea el prompt con el estilo deseado, insertando la variable 'cancion'
# Se crea una lista con un solo string para que el tokenizer procese un batch de tamaño 1
# Luego se convierte el texto a tensores de PyTorch y se pasa a la GPU
inputs = tokenizer([promp_style.format(cancion)], return_tensors="pt").to('cuda')

# Genera texto a partir del prompt
outputs = model.generate(
    input_ids=inputs.input_ids,           # IDs de entrada tokenizados
    attention_mask=inputs.attention_mask, # Máscara de atención para que el modelo ignore los tokens de relleno
    max_new_tokens=2,                  # Número máximo de tokens nuevos a generar
    use_cache=True,                       # Usa caché para acelerar la generación
    eos_token_id=tokenizer.eos_token_id,
)

# Decodifica SOLO los tokens generados (excluye el prompt)
prompt_length = inputs.input_ids.shape[1]
generated_tokens = outputs[:, prompt_length:]

# skip_special_tokens=True es VITAL para eliminar <|begin_of_sentence|>, etc.
# clean_up_tokenization_spaces=True puede ayudar con espacios extra.
decoded_outputs = tokenizer.batch_decode(
    generated_tokens,
    skip_especial_tokens=True,
    clean_up_tokenization_spaces=True
)

if not decoded_outputs:
    print("Error: La decodificación no produjo ninguna salida.")
    final_answer = None # O manejar el error
else:
    # Puede que necesites ajustar esto si la salida siempre está fragmentada en la lista
    # Por ejemplo, si siempre es el penúltimo o último elemento no vacío:
    # response_text = "".join([s for s in decoded_outputs if s.strip()]) # Une todo lo no vacío
    response_text = decoded_outputs[0].strip() # Toma el primer resultado decodificado y quita espacios extra
    print(f"--- Texto Generado Decodificado (limpio) ---\n{response_text}\n------------------------------------------")

    final_answer = None
    found_method = "None"

    # Prioridad 1: Buscar después de </think>
    # Busca la etiqueta literal "</think>" seguida de cualquier cantidad de espacio en blanco (\s*)
    # y luego captura el primer dígito que encuentre ([01]).
    match_think = re.search(r"</think>\s*([01])", response_text)
    if match_think:
        final_answer = match_think.group(1)
        found_method = "After </think>"

    # Prioridad 2: Buscar después de "Respuesta:" (si la Prioridad 1 falló)
    # Esto es por si el modelo *genera* "Respuesta:" en su salida, aunque es menos probable con el prompt actual.
    if final_answer is None:
        # Busca "Respuesta" (case-insensitive) seguido de ':' opcional, espacios, y captura el dígito.
        match_respuesta = re.search(r"Respuesta:?\s*([01])", response_text, re.IGNORECASE)
        if match_respuesta:
            final_answer = match_respuesta.group(1)
            found_method = "After 'Respuesta:' (in output)" # Indica que se encontró en la salida

    # Prioridad 3: Fallbacks (si las Prioridades 1 y 2 fallaron)
    if final_answer is None:
        print("INFO: No se encontró patrón '</think> [0/1]' ni 'Respuesta: [0/1]'. Intentando fallbacks...")
        # Fallback A: Último dígito no-espacio
        cleaned_end_text = response_text.rstrip()
        if cleaned_end_text and cleaned_end_text[-1] in ('0', '1'):
            final_answer = cleaned_end_text[-1]
            found_method = "Fallback: Last non-whitespace character"
        # Fallback B: Primer dígito (solo si Fallback A falló)
        elif response_text and response_text.startswith(('0', '1')):
             # Usamos el texto ya limpiado con strip() al principio
             final_answer = response_text[0]
             found_method = "Fallback: First character"

    # Resultado Final
    if final_answer is None:
        final_answer = '?' # O None
        found_method = "Failed to find digit"
        print("ERROR: No se pudo extraer '0' o '1' de forma fiable.")
        # Considera registrar 'response_text' completo para depuración
    elif final_answer not in ('0', '1'): # Doble chequeo por si acaso
         print(f"ERROR INESPERADO: Se extrajo algo que no es '0' o '1': {final_answer}")
         final_answer = '?'
         found_method += " (Extraction Error)"


print(f"\nExtraction Method Used: {found_method}")
print(f"Respuesta Final Procesada: {final_answer}")
# Decodifica los tokens generados a texto legible
#response = tokenizer.batch_decode(outputs)

# AHORA EMPIEZA EL FINE TUNING

# LLAMADO "ALPACA PROMPT"

In [ ]:
prompts_train = [ """
            ### Instruccion
            Analiza la siguiente letra de canción y clasifícala en UNA de las siguientes cuatro categorías: Sexualización, Odio, Violencia, o No relacionado.

            Evalúa si la letra contiene predominantemente:
              - Violencia: Descripciones explícitas de agresión física, uso de armas, peleas, asesinatos, daño físico, amenazas directas, o glorificación de la violencia física.
              - Odio: Expresiones de hostilidad intensa, desprecio profundo, animadversión, lenguaje deshumanizante o extremadamente denigrante hacia individuos o grupos (por cualquier motivo: rivalidad, raza, género, etc.), o incitación al rencor/venganza.
              - Sexualizacion: Reducción de personas a objetos sexuales, enfoque excesivo en atributos físicos de forma despersonalizada, descripciones sexuales explícitas con fin de objetivar/degradar, o lenguaje sexualmente sugerente de manera cosificadora.
              - No relacionado: Si la letra NO contiene predominantemente ninguno de los elementos anteriores (puede ser romántica, triste, festiva, narrativa sin estos enfoques, etc.).

            Piensa cuidadosamente tu respuesta paso a paso, considerando cada categoría. Determina cuál es la categoría principal o más definitoria si hubiera solapamiento leve. Responde únicamente con la etiqueta de la categoría elegida: "Violencia", "Odio", "Sexualizacion", o "No relacionado". No proporciones ninguna explicación ni texto adicional fuera de la etiqueta final.

            ### Letra:
            {lyrics}

            ### Respuesta:
            <think> {reasoning} </think>{label}""",
          ]

In [ ]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

In [ ]:
label_map = {
    0: "No relacionado",
    1: "Sexualizacion",
    2: "Violencia",
    3: "Odio"
}

def formatting_prompts(examples):
    # Extrae las listas de letras y etiquetas desde el diccionario de ejemplos
    lyrics_list = examples[COLUMN_LETRA]      # Lista de letras (inputs)
    labels_list  = examples[COLUMN_ETIQUETA]      # Lista de etiquetas o salidas esperadas
    reasonings_list = examples[COLUMN_RAZONAMIENTO]

    texts = []  # Aquí se guardarán los prompts formateados


    # Itera sobre cada par input/output
    for lyric, label, reasoning in zip(lyrics_list, labels_list, reasonings_list):
        label_str = label_map[label]
        # Formatea el prompt usando la plantilla
        text = prompts_train[tipo_prompt].format(
            lyrics=str(lyric),
            reasoning=str(reasoning),
            label=label_str
            ) + EOS_TOKEN  # Añade token de finalización

        texts.append(text)  # Guarda el texto formateado

    # Devuelve un diccionario con el texto ya preparado para el entrenamiento
    return {
        "text": texts
    }

In [ ]:
print("\nFormateando datasets para entrenamiento CoT...")
print(f"Columnas en trainData antes de mapear: {trainData.column_names}")
print(f"Columnas en validData antes de mapear: {validData.column_names}")

In [ ]:
dataset_finetune = trainData.map(formatting_prompts, batched = True, remove_columns=trainData.column_names) # remove_columns para dejar solo 'text'
eval_finetune = validData.map(formatting_prompts, batched=True, remove_columns=validData.column_names) # remove_columns para dejar solo 'text'
dataset_finetune["text"][0]

# Calculo longitud de los prompts

In [ ]:
import matplotlib.pyplot as plt

token_lengths = [len(tokenizer.encode(text, add_special_tokens=False)) for text in dataset_finetune["text"]]
print(f"Se calcularon las longitudes de {len(token_lengths)} prompts formateados.")

if not token_lengths:
    print("La lista de longitudes está vacía. Verifica el dataset y la columna 'text'.")
else:
    # --- Ahora puedes proceder con el análisis ---

    # 1. Encontrar la longitud máxima
    max_len = max(token_lengths)
    print(f"Longitud máxima encontrada (incl. special tokens): {max_len}")

    # 2. Visualizar la distribución (como en tu ejemplo anterior)
    plt.figure(figsize=(12, 7))
    plt.hist(token_lengths, bins=50, alpha=0.7, label=f'Distribución (Máx={max_len})')

    plt.axvline(1024, color='red', linestyle='dashed', linewidth=1, label='Max Length = 1024')
    plt.axvline(2048, color='orange', linestyle='dashed', linewidth=1, label='Max Length = 2048')
    plt.axvline(4096, color='green', linestyle='dashed', linewidth=1, label='Max Length = 4096')
    plt.axvline(8192, color='blue', linestyle='dashed', linewidth=1, label='Max Length = 8192')


    plt.xlabel('Número de Tokens (incl. special tokens)')
    plt.ylabel('Número de Ejemplos')
    plt.title(f'Distribución de Longitud de Tokens del Prompt Formateado\n(Modelo: {tokenizer.name_or_path})') # Usa el nombre del tokenizer
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    print(f"Porcentaje de canciones conS más de 1024 tokens: {sum(l > 1024 for l in token_lengths) / len(token_lengths) * 100:.2f}%")
    print(f"Porcentaje de canciones con más de 2048 tokens: {sum(l > 2048 for l in token_lengths) / len(token_lengths) * 100:.2f}%")
    print(f"Porcentaje de canciones con más de 4096 tokens: {sum(l > 4096 for l in token_lengths) / len(token_lengths) * 100:.2f}%")
    print(f"Porcentaje de canciones con más de 8192 tokens: {sum(l > 8192 for l in token_lengths) / len(token_lengths) * 100:.2f}%")

# ELIMINO LOS PROMPTS QUE SUPEREN LOS 2048 TOKENS

In [ ]:
def limit_tokens(example):
  return len(tokenizer.encode(str(example['text']), add_special_tokens=True)) <= max_seq_length

In [ ]:
print(f"Tamaño original de dataset_finetune: {len(dataset_finetune)}")
print(f"Tamaño original de eval_finetune: {len(eval_finetune)}")
dataset_finetune = dataset_finetune.filter(limit_tokens, batched=False)
eval_finetune = eval_finetune.filter(limit_tokens, batched=False)
print(f"Tamaño filtrado de dataset_finetune: {len(dataset_finetune)}")
print(f"Tamaño filtrado de eval_finetune: {len(eval_finetune)}")

In [ ]:
import matplotlib.pyplot as plt

token_lengths = [len(tokenizer.encode(text, add_special_tokens=False)) for text in dataset_finetune["text"]]
print(f"Se calcularon las longitudes de {len(token_lengths)} prompts formateados.")

if not token_lengths:
    print("La lista de longitudes está vacía. Verifica el dataset y la columna 'text'.")
else:
    # --- Ahora puedes proceder con el análisis ---

    # 1. Encontrar la longitud máxima
    max_len = max(token_lengths)
    print(f"Longitud máxima encontrada (incl. special tokens): {max_len}")

    # 2. Visualizar la distribución (como en tu ejemplo anterior)
    plt.figure(figsize=(12, 7))
    plt.hist(token_lengths, bins=50, alpha=0.7, label=f'Distribución (Máx={max_len})')

    plt.axvline(1024, color='red', linestyle='dashed', linewidth=1, label='Max Length = 1024')
    plt.axvline(2048, color='orange', linestyle='dashed', linewidth=1, label='Max Length = 2048')
    plt.axvline(4096, color='green', linestyle='dashed', linewidth=1, label='Max Length = 4096')
    plt.axvline(8192, color='blue', linestyle='dashed', linewidth=1, label='Max Length = 8192')


    plt.xlabel('Número de Tokens (incl. special tokens)')
    plt.ylabel('Número de Ejemplos')
    plt.title(f'Distribución de Longitud de Tokens del Prompt Formateado\n(Modelo: {tokenizer.name_or_path})') # Usa el nombre del tokenizer
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

    print(f"Porcentaje de canciones con más de 1024 tokens: {sum(l > 1024 for l in token_lengths) / len(token_lengths) * 100:.2f}%")
    print(f"Porcentaje de canciones con más de 2048 tokens: {sum(l > 2048 for l in token_lengths) / len(token_lengths) * 100:.2f}%")
    print(f"Porcentaje de canciones con más de 4096 tokens: {sum(l > 4096 for l in token_lengths) / len(token_lengths) * 100:.2f}%")
    print(f"Porcentaje de canciones con más de 8192 tokens: {sum(l > 8192 for l in token_lengths) / len(token_lengths) * 100:.2f}%")

# Parametros del train

In [ ]:
# --- Ajustes para 14GB VRAM y cálculo de pasos ---
per_device_batch = 1
gradient_accum_steps = 16 # Si da OOM, bajar a 4.
effective_batch_size = per_device_batch * gradient_accum_steps # = 8
epochs = 8
num_train_examples = 843 # Tamaño de tu dataset de entrenamiento
steps_per_epoch = num_train_examples // effective_batch_size # ~177
total_steps = steps_per_epoch * epochs # ~296
warmup_steps_calculated = int(total_steps * 0.10) # ~30 (10% warmup)
# Evaluar/Loguear/Guardar cada 25 pasos
eval_logging_save_steps = 25
lora_r = 16
lora_alpha = 32
learning_rate = 1e-5

print(f"Configuración de Entrenamiento:")
print(f"  - Per Device Batch Size: {per_device_batch}")
print(f"  - Gradient Accumulation Steps: {gradient_accum_steps}")
print(f"  - Effective Batch Size: {effective_batch_size}")
print(f"  - Num Train Examples: {num_train_examples}")
print(f"  - Epochs: {epochs}")
print(f"  - Steps per Epoch: ~{steps_per_epoch}")
print(f"  - Total Steps: ~{total_steps}")
print(f"  - Warmup Steps: {warmup_steps_calculated}")
print(f"  - Eval/Log/Save Steps: {eval_logging_save_steps}")
print(f"  - LoRA R: {lora_r}")
print(f"  - LoRA Alpha: {lora_alpha}")

In [ ]:
# Aplica LoRA al modelo base para permitir entrenamiento eficiente
model_lora = FastLanguageModel.get_peft_model(
    model,                          # Modelo base al que se le quiere aplicar LoRA
    r=lora_r,                           # Rango bajo de adaptación (más bajo = menos parámetros entrenables)
    target_modules=[               # Capas específicas del modelo donde se insertarán los adaptadores LoRA
        "q_proj", "k_proj", "v_proj", "o_proj",        # Proyecciones de atención
        "gate_proj", "up_proj", "down_proj"           # Capas del MLP
    ],
    lora_alpha=lora_alpha,                 # Escala del adaptador LoRA (afecta la fuerza de la adaptación)
    lora_dropout=0.05,                # Dropout en LoRA (0 = no dropout, útil para tener resultados estables)
    bias="none",                   # No se entrena el sesgo de las capas originales
    use_gradient_checkpointing="unsloth",  # Reduce el uso de memoria en GPU durante el entrenamiento
    random_state=3407,             # Fijar semilla aleatoria para reproducibilidad
    use_rslora=False,              # Si se quiere usar una versión más avanzada de LoRA (aquí se desactiva)
    loftq_config=None              # Solo se usa si se quiere aplicar cuantización LoFTQ (aquí no se usa)
)

In [ ]:
#Inicializar el fine tuning trainer
trainer = SFTTrainer(
    model = model_lora,                 # El modelo con adaptadores LoRA listos para entrenar
    tokenizer = tokenizer,              # El tokenizer asociado al modelo
    train_dataset = dataset_finetune,   # Dataset ya formateado con prompts
    eval_dataset = eval_finetune,
    dataset_text_field = "text",        # Campo del dataset que contiene el texto de entrada y salida
    max_seq_length = max_seq_length,    # Longitud máxima de secuencia para cada ejemplo
    dataset_num_proc = 1,               # Número de procesos paralelos para preparar el dataset
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
    args = TrainingArguments(
        per_device_train_batch_size = per_device_batch,       # Tamaño de batch por GPU (se combina con gradient_accumulation_steps)
        gradient_accumulation_steps = gradient_accum_steps,       # Acumula gradientes durante 4 pasos antes de actualizar pesos → batch efectivo = 8
        #max_steps=60,
        num_train_epochs = epochs,                  # Número de épocas completas sobre el dataset
        warmup_steps = warmup_steps_calculated,                      # Pasos de warmup donde la LR va subiendo poco a poco
        learning_rate = learning_rate,                  # Tasa de aprendizaje (bastante buena para LoRA)
        fp16 = not is_bfloat16_supported(),    # Usa FP16 si bfloat16 no está soportado
        bf16 = is_bfloat16_supported(),        # Usa BF16 si tu GPU lo soporta (mejor para Ampere y posteriores)
        logging_steps = eval_logging_save_steps,                    # Registra logs cada 10 pasos
        #logging_steps = 1,
        optim = "adamw_8bit",                  # Optimizador AdamW en 8 bits (menos memoria, ideal para GPUs pequeñas)
        weight_decay = 0.01,                   # Regularización para evitar overfitting
        lr_scheduler_type = "linear",          # Scheduler lineal de tasa de aprendizaje
        seed = 3407,                           # Semilla para reproducibilidad
        output_dir = f"{path}/cuadernos/outputs",                # Carpeta donde se guardan los checkpoints
        eval_strategy="steps",
        eval_steps=eval_logging_save_steps,
        save_strategy="steps",
        save_steps=eval_logging_save_steps,
        save_total_limit=4,                                   # Guardar solo los últimos 2
        load_best_model_at_end=True,                          # Carga el mejor modelo según eval_loss
        metric_for_best_model="eval_loss",                    # Usa loss para decidir el "mejor"
        greater_is_better=False,                              # Menor loss es mejor
        report_to="wandb",
    ),
)

#Iniciando training

In [ ]:
# --- Ejecutar Entrenamiento ---
print("\nIniciando entrenamiento...")
trainer_stats = trainer.train()
print("Entrenamiento completado.")

In [ ]:
trainer_stats = trainer.train(resume_from_checkpoint = True)

In [ ]:
wandb.finish()

#Guardando el modelo

In [ ]:
final_save_path = f'{path}/models/Model_{model_name}-CoT_{tipo_CoT}-R_{lora_r}-Alpha_{lora_alpha}-LR_{learning_rate}-Tarea_2'

In [ ]:

print(f"\nGuardando el mejor modelo en: {final_save_path}")
trainer.save_model(final_save_path)
tokenizer.save_pretrained(final_save_path)
print("Modelo y tokenizer guardados.")

# Probando el modelo entrenado :)

In [ ]:
final_save_path = f'{path}/models/Model-DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit-CoT-GPT4o-R-16-Alpha-32-LR-2e-05-Tarea-2'

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = final_save_path,                         # Nombre del modelo en Hugging Face
    max_seq_length = 2048,                      # Longitud máxima de secuencia que puede procesar
    dtype = dtype,                                        # Tipo de precisión para los cálculos (torch.float16 o torch.bfloat16)
)

In [ ]:
prompts_test = [ """
            ### Instruccion
            Analiza la siguiente letra de canción y clasifícala en UNA de las siguientes cuatro categorías: Sexualización, Odio, Violencia, o No relacionado.

            Evalúa si la letra contiene predominantemente:
              - Violencia: Descripciones explícitas de agresión física, uso de armas, peleas, asesinatos, daño físico, amenazas directas, o glorificación de la violencia física.
              - Odio: Expresiones de hostilidad intensa, desprecio profundo, animadversión, lenguaje deshumanizante o extremadamente denigrante hacia individuos o grupos (por cualquier motivo: rivalidad, raza, género, etc.), o incitación al rencor/venganza.
              - Sexualizacion: Reducción de personas a objetos sexuales, enfoque excesivo en atributos físicos de forma despersonalizada, descripciones sexuales explícitas con fin de objetivar/degradar, o lenguaje sexualmente sugerente de manera cosificadora.
              - No relacionado: Si la letra NO contiene predominantemente ninguno de los elementos anteriores (puede ser romántica, triste, festiva, narrativa sin estos enfoques, etc.).

            Piensa cuidadosamente tu respuesta paso a paso, considerando cada categoría. Determina cuál es la categoría principal o más definitoria si hubiera solapamiento leve. Responde únicamente con la etiqueta de la categoría elegida: "Violencia", "Odio", "Sexualizacion", o "No relacionado". No proporciones ninguna explicación ni texto adicional fuera de la etiqueta final.

            ### Letra:
            {lyrics}

            ### Respuesta:
            <think>"""
          ]

In [ ]:
# --- Asegurar que el modelo esté en modo evaluación e inferencia ---
model.eval() # Pone el modelo en modo evaluación (desactiva dropout, etc.)
FastLanguageModel.for_inference(model) # Prepara el modelo Unsloth para inferencia rápida

#Prueba para ver los FP y FN

#Prueba para ejecutar solo 40 canciones

In [ ]:
import random
# --- SELECCIÓN DEL SUBCONJUNTO DE testData ---

num_samples_label_1 = 6  # Número de ejemplos misóginos a seleccionar
num_samples_label_0 = 6  # Número de ejemplos no misóginos a seleccionar

# 1. Separar los ejemplos por etiqueta
examples_label_1 = [ex for ex in testData if str(ex[COLUMN_ETIQUETA]).strip() == '2']
examples_label_0 = [ex for ex in testData if str(ex[COLUMN_ETIQUETA]).strip() == '3']

# 2. Asegurarse de no pedir más ejemplos de los que hay
num_samples_label_1 = min(num_samples_label_1, len(examples_label_1))
num_samples_label_0 = min(num_samples_label_0, len(examples_label_0))

# 3. Seleccionar aleatoriamente los ejemplos deseados
random.seed(42) # Para reproducibilidad
selected_examples_1 = random.sample(examples_label_1, num_samples_label_1)
selected_examples_0 = random.sample(examples_label_0, num_samples_label_0)

# 4. Combinar los ejemplos seleccionados en un nuevo dataset para evaluar
sampled_testData = selected_examples_1 + selected_examples_0
random.shuffle(sampled_testData) # Mezclar las clases 0 y 1 para la evaluación

testData = sampled_testData
print(f"--- Preparando evaluación con un subconjunto ---")
print(f"Seleccionados {len(selected_examples_1)} ejemplos con etiqueta '1'.")
print(f"Seleccionados {len(selected_examples_0)} ejemplos con etiqueta '0'.")
print(f"Total ejemplos en el subconjunto de evaluación: {len(sampled_testData)}")

#Ejecutar evaluación

In [ ]:
# --- Listas para guardar resultados ---
true_labels = []
predicted_labels = []
failed_extractions_count = 0
test_results_details = []

from tqdm import tqdm
import re
print(f"\n--- Iniciando evaluación en el Test Set ({len(testData)} ejemplos) ---")

for example in tqdm(testData, desc="Evaluando Test Set"):
    try:
        lyrics = str(example[COLUMN_LETRA])
        true_label = str(example[COLUNM_ETIQUETALETRA]).strip()

        if true_label == 'NR':
            text_label = 'No relacionado'
        elif true_label == 'V':
            text_label = 'Violencia'
        elif true_label == 'H':
            text_label = 'Odio'
        elif true_label == 'S':
            text_label = 'Sexualizacion'

        # 1. Formatear el prompt de inferencia
        input_text = prompts_test[tipo_prompt].format(lyrics=str(lyrics))

        # 2. Tokenizar
        inputs = tokenizer([input_text], return_tensors="pt", truncation=True, max_length=max_seq_length).to(device)

        pred_label = None
        retries = 0

        while retries <= 4 and pred_label not in ["NR", "V", "H", "S"]:
            if retries > 0:
                print(f'---Reintento {retries}/4 para ID: {example[COLUMN_ID]}---')
            # 3. Generar la salida del modelo
            with torch.no_grad():
                outputs = model.generate(
                    input_ids=inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    temperature = 0.6,
                    max_new_tokens=1024,  # Suficiente para CoT + </think> + label
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id,
                    do_sample=False,     # Forzar salida determinista para evaluación
                    use_cache=True,
                    repetition_penalty=1.1,
                )

            # 4. Decodificar SOLO los tokens generados
            prompt_length = inputs.input_ids.shape[1]
            generated_ids = outputs[0, prompt_length:]
            decoded_output = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            print("-----------------------------------------------------------------------")
            print("ID "+example[COLUMN_ID])
            print("True label " + text_label)
            print("Salida\n" + decoded_output)
            target_labels = ["Sexualizacion", "Odio", "Violencia", "No relacionado"]
            regex_pattern = r"(?is)<\/think>\s*(.*)"
            #regex_pattern = r"(?i)</think>\s*(" + "|".join(target_labels) + r")\s*$"
            match = re.search(regex_pattern, decoded_output)
            if match:
                pred_label = match.group(1).strip().capitalize()
            if pred_label not in target_labels:
                pred_label = None  # Forzar otro intento
                if retries == 4:
                    pred_label = 'No relacionado'
                retries += 1
                continue  # Ir al siguiente intento


            if pred_label == 'No relacionado':
                pred_label = 'NR'
            elif pred_label == 'Violencia':
                pred_label = 'V'
            elif pred_label == 'Odio':
                pred_label = 'H'
            elif pred_label == 'Sexualizacion':
                pred_label = 'S'

            # 6. Almacenar resultados si la extracción fue válida
            if pred_label in ('NR', 'V', 'H', 'S'):
                true_labels.append(true_label)
                predicted_labels.append(pred_label)
                test_results_details.append({
                    'id': example[COLUMN_ID],
                    'lyrics': lyrics,
                    'true_label': true_label,
                    'predicted_label': pred_label,
                    'full_output_decoded': decoded_output # Guardar salida completa para análisis
                })

    except Exception as e:
        print(f"\nError procesando un ejemplo durante la evaluación: {e}")
        failed_extractions_count += 1 # Contarlo como fallo

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# --- Calcular y mostrar métricas ---
print(f"\n--- Resultados de la Evaluación (Test Set) ---")
print(f"Total de ejemplos evaluados: {len(true_labels)}")
print(f"Ejemplos donde falló la extracción de la etiqueta: {failed_extractions_count}")

class_labels = ['NR', 'S', 'V', 'H']
class_labels_display = ["No relacionado (NR)", "Sexualizacion (S)", "Violencia (V)", "Odio (H)"]

if not true_labels or not predicted_labels:
    print("\nNo se pudieron calcular las métricas (no hay predicciones válidas).")
else:
    # Calcular métricas usando sklearn
    accuracy = accuracy_score(true_labels, predicted_labels)
    f1_macro = f1_score(true_labels, predicted_labels, average='macro', zero_division=0)
    f1_weighted = f1_score(true_labels, predicted_labels, average='weighted', zero_division=0)
    report = classification_report(true_labels, predicted_labels, labels=class_labels, target_names=class_labels_display, zero_division=0)
    conf_matrix = confusion_matrix(true_labels, predicted_labels, labels=class_labels)

    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"F1 Score (Macro): {f1_macro:.4f}")
    print(f"F1 Score (Weighted): {f1_weighted:.4f}")
    print("\nReporte de Clasificación Detallado:")
    print(report)

    # Grafico de la matriz de confusión
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Oranges', xticklabels=class_labels_display, yticklabels=class_labels_display)
    plt.ylabel('Etiqueta verdadera')
    plt.xlabel('Etiqueta predecida')
    plt.title('Matriz de confusión Tarea 2')
    matriz_path = f"{path}/matrices/prueba.png"
    plt.savefig(matriz_path)
    plt.show()